# Minimising computation time

`AutoEmulate` can be slow if the input data has many observations (rows) or many output variables. By default, `AutoEmulate` cross-validates each model, so we're computing 5 fits per models. The computation time will be relatively short for datasets up to a few thousands of datapoints, but some models (e.g. Gaussian Processes) don't scale well, so computation time might quickly become an issue. 

In this tutorial we walk through four strategies to speed up `AutoEmulate`:

1) parallelise model fits using `n_jobs` 
2) restrict the range of models using the `models` argument 
3) run fewer cross validation folds using `cross_validator` 
4) for hyperparameter search:
    - all of the above
    - run fewer iterations using `param_search_iters`

In [1]:
from sklearn.datasets import make_regression
from autoemulate.compare import AutoEmulate

Let's make a dataset.

In [2]:
X, y = make_regression(n_samples=500, n_features=10, n_targets=5)
X.shape, y.shape

((500, 10), (500, 5))

And see how long `AutoEmulate` takes to run (without hyperparameter search).

In [3]:
import time

start = time.time()

em = AutoEmulate()
em.setup(X, y)
em.compare()

end = time.time()
print(f"Time taken: {end - start} seconds")

,Values
Simulation input shape (X),"(500, 10)"
Simulation output shape (y),"(500, 5)"
Proportion of data for testing (test_set_size),0.2
Scale input data (scale),True
Scaler (scaler),StandardScaler
Scale output data (scale_output),True
Scaler output (scaler_output),StandardScaler
Do hyperparameter search (param_search),False
Reduce input dimensionality (reduce_dim),False
Reduce output dimensionality (reduce_dim_output),False


Cross-validating:   0%|          | 0/8 [00:00<?, ?it/s]

Time taken: 35.2008638381958 seconds


## 1) parallise model fits using `n_jobs`
The n_jobs parameter allows you to specify the number of CPU cores to use for parallel processing. Setting n_jobs = -1  uses all available cores, speeding up computations when working with large datasets.

Note: Maxing out all available cores might not always lead to faster computation times. Due to overhead from parallelization, memory bandwidth limitations, and potential load imbalances, using more cores can sometimes result in diminishing returns or even slower performance.

Here we accomplish a speed-up by setting n_jobs to 5.

In [4]:
start = time.time()

em = AutoEmulate()
em.setup(X, y, n_jobs=5, print_setup=False)
em.compare()

end = time.time()
print(f"Time taken: {end - start} seconds")

Cross-validating:   0%|          | 0/8 [00:00<?, ?it/s]

Time taken: 51.6426100730896 seconds


## 2) restrict the range of models

Another approach is to limit the range of models by selecting a subset of relevant types based on your domain and problem expertise. This selection process typically considers factors such as the nature of the problem, data characteristics or the need for interpretability. By narrowing down the types of models, you can reduce the computational burden and focus on the most promising architectures for your specific task.

In [5]:
em = AutoEmulate()
em.setup(X, y, print_setup=False)

# let's see all models, which we can refer to by short or full name
em.model_registry.get_model_names()

# setup with fewer models
start = time.time()

em.setup(X, y, models=["sop", "rbf", "gb"], print_setup=False)
em.compare()

end = time.time()
print(f"Time taken: {end - start} seconds")

Cross-validating:   0%|          | 0/3 [00:00<?, ?it/s]

Time taken: 2.5493457317352295 seconds


## 3) reduce the number of folds in cross validation using `cross_validator` 

With larger datasets, you might initially want to set the number of folds for the cross validation to 3 instead of 5 (the default), so that there are fewer models to fit. `AutoEmulate` takes a `cross_validator` argument, which takes an scklearn cross validator or [splitter](https://scikit-learn.org/stable/api/sklearn.model_selection.html). Let's use kfold with 3 splits, which saves 2 model fits per model.

In [6]:
from sklearn.model_selection import KFold

start = time.time()

em = AutoEmulate()
em.setup(X, y, cross_validator=KFold(n_splits=3), print_setup=False)
em.compare()

end = time.time()
print(f"Time taken: {end - start} seconds")

Cross-validating:   0%|          | 0/8 [00:00<?, ?it/s]

Time taken: 18.824736833572388 seconds


## 4) modify hyperparameter search

If we want to use hyperparameter search, we suddenly have to fit many more models. For each model, we might have 20 different parameter combinations, and because we cross validate each combination, we are running 20 * 5 = 100 model fits per model. It's therefore recommended to focus on a few models of interest when using hyperparameter search.

To get a ballpark figure for how long hyperparameter search might take, we can run `AutoEmulate` without hyperparameter search, and then multiply the time taken by the number of parameter combinations we want to try.

In [7]:
start = time.time()

em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()

end = time.time()
run_time = end - start
print(f"Time taken: {run_time} seconds")

Cross-validating:   0%|          | 0/8 [00:00<?, ?it/s]

Time taken: 29.96882200241089 seconds


The default number parameter combinations to search over (see`param_search_iters`) is 20, so we can expect hyperparameter search to take `20 * run_time` seconds. Although this can be sped up by running in parallel or running fewer cross-validation folds, we are usually interested only in some emulator models anyway, which will speed up computation time. To figure out which models to optimise, let's inspect the cv results from the training data.

In [8]:
em.summarise_cv()

,preprocessing,model,short,fold,rmse,r2
0,None,SecondOrderPolynomial,sop,2,0.000009,1.000000
1,None,SecondOrderPolynomial,sop,4,0.000009,1.000000
2,None,RadialBasisFunctions,rbf,2,0.000010,1.000000
3,None,RadialBasisFunctions,rbf,4,0.000009,1.000000
4,None,SecondOrderPolynomial,sop,1,0.000011,1.000000
5,None,RadialBasisFunctions,rbf,0,0.000010,1.000000
6,None,SecondOrderPolynomial,sop,0,0.000010,1.000000
7,None,RadialBasisFunctions,rbf,1,0.000011,1.000000
8,None,SecondOrderPolynomial,sop,3,0.000012,1.000000
9,None,RadialBasisFunctions,rbf,3,0.000012,1.000000


Now, we might like to see whether the Support Vector Machines model could do better with better hyperparameters. To minimise computation time, we only take those this one model, and run 50 iterations (instead of the default 20), we run them in parallel using 5 jobs.

In [9]:
start = time.time()

em_svm = AutoEmulate()
em_svm.setup(X, y, models=["svm"], param_search=True, param_search_iters=50, 
         n_jobs=5, print_setup=False)
em_svm.compare()

end = time.time()
print(f"Time taken: {end - start} seconds")

em_svm.summarise_cv()

Optimising and cross-validating:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken: 6.434678077697754 seconds


,preprocessing,model,short,fold,rmse,r2
0,None,SupportVectorMachines,svm,2,8.720925,0.997536
1,None,SupportVectorMachines,svm,0,7.939373,0.997482
2,None,SupportVectorMachines,svm,1,9.151469,0.997225
3,None,SupportVectorMachines,svm,3,9.114729,0.997204
4,None,SupportVectorMachines,svm,4,8.731051,0.997105


And indeed, the SVM model does perform better with better hyperparameters! Let's have a look at them. SVM is a pipeline, so we first extract the 'model' step, and then get the parameters. 

In [10]:
svm = em_svm.get_model(name="svm")
svm.named_steps['model'].get_params()

{'estimator__C': 2.9745514805217894,
 'estimator__cache_size': 200,
 'estimator__coef0': 0.6864724407544227,
 'estimator__degree': 4,
 'estimator__epsilon': 0.11043299726789946,
 'estimator__gamma': 'auto',
 'estimator__kernel': 'linear',
 'estimator__max_iter': -1,
 'estimator__normalise_y': True,
 'estimator__shrinking': False,
 'estimator__tol': 0.0010043197053019159,
 'estimator__verbose': False,
 'estimator': SupportVectorMachines(C=2.9745514805217894, coef0=0.6864724407544227, degree=4,
                       epsilon=0.11043299726789946, gamma='auto',
                       kernel='linear', shrinking=False,
                       tol=0.0010043197053019159),
 'n_jobs': None}